In [1]:
#import ultralytics
#ultralytics.checks()
#from ultralytics import YOLO

#import dayolo
#from dayolo import YOLO

import yolo
from yolo import YOLO

from PIL import Image
import torch
import yaml

import os
import random
#from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

#import visualization_utils as visutils

/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
#pretrained_model_name = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#model_path = 'runs/detect/' + pretrained_model_name + '/weights/best.pt'
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO("yolov8.yaml").load("yolov8m.pt")
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO('yolov8m.pt', task='detect')
#model = YOLO("yolov8m.yaml", task='detect').load("yolov8m.pt")
model = YOLO("yolov8m.yaml", task='detect', subtask='unsuperviseddomainclassifier').load("yolov8m.pt")


#model.load_weights('yolov8m.pt')


#PRETRAINED_MODEL_NAME = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#PRETRAINED_MODEL_PATH = 'src/model/runs/detect/' + PRETRAINED_MODEL_NAME + '/weights/best.pt'

#MODEL_NAME = 'pfeifer_penguins_poland_palmyra_10percent_bckgd_yolov8m_120epochs'
#TASK = 'detect' # Choose between: 'deepcoral_detect' 'detect'
#model = YOLO('yolov8m.pt')

INITIALIZING DOMAIN CLASSIFIER MODEL

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  yolo.nn.modules.conv.Conv                    [3, 48, 3, 2]                 
  1                  -1  1     41664  yolo.nn.modules.conv.Conv                    [48, 96, 3, 2]                
  2                  -1  2    111360  yolo.nn.modules.block.C2f                    [96, 96, 2, True]             
  3                  -1  1    166272  yolo.nn.modules.conv.Conv                    [96, 192, 3, 2]               
  4                  -1  4    813312  yolo.nn.modules.block.C2f                    [192, 192, 4, True]           
  5                  -1  1    664320  yolo.nn.modules.conv.Conv                    [192, 384, 3, 2]              


  6                  -1  4   3248640  yolo.nn.modules.block.C2f                    [384, 384, 4, True]           
  7                  -1  1   1991808  yolo.nn.modules.conv.Conv                    [384, 576, 3, 2]              
  8                  -1  2   3985920  yolo.nn.modules.block.C2f                    [576, 576, 2, True]           
  9                  -1  1    831168  yolo.nn.modules.block.SPPF                   [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  yolo.nn.modules.conv.Concat                  [1]                           
 12                  -1  2   1993728  yolo.nn.modules.block.C2f                    [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  yolo.nn.modules.conv.Concat                  [1]  

In [4]:
print(model.task)
print(model.subtask)

detect
domainclassifier


In [5]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 24:
        print("block", i)
        print(a)

block 24
AdaptiveAvgPooling(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (fc1): Linear(in_features=1, out_features=2, bias=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
)


In [6]:
# TRAIN the model on our dataset (data.yml config file) 

model_name = 'TEST_features_loss'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=2,
   batch=8, #32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=10,
   device=0,
   verbose=True,
   val = True,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, scale=0.5, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.1.6 available 😃 Update with 'pip install -U ultralytics'
New https://pypi.org/project/ultralytics/8.1.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=data.yaml, epochs=2, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=TEST_features_loss18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, source_name=global_birds_palmyra, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, ma

train: Scanning /gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/labels.cache... 1722 images, 182 backgrounds, 0 corrupt: 100%|██████████| 1904/1904 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/val/labels.cache... 248 images, 19 backgrounds, 0 corrupt: 100%|██████████| 267/267 [00:00<?, ?it/s]
/home/eec42/.conda/envs/pdm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Plotting labels to runs/detect/TEST_features_loss18/labels.jpg... 
Plotting labels to runs/detect/TEST_features_loss18/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 80 weight(decay=0.0), 88 weight(decay=0.0005), 90 bias(decay=0.0)
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 80 weight(decay=0.0), 88 weight(decay=0.0005), 90 bias(decay=0.0)
2 epochs...
2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss    da_loss  Instances       Size

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss    da_loss  Instances       Size


...

        1/2      3.67G      5.267      10.59      3.915     0.7009         45        640:  30%|██▉       | 71/238 [00:09<00:18,  9.18it/s]

batch ['/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_896_patch_380.0_380.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_569_patch_0.0_0.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_751_patch_0.0_760.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_penguins_offshore_rocks_cape_wallace_survey_4_1561_patch_0.0_0.0_480_480.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_72_patch_380.0_760.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_penguins_offshore_rocks_cape_wallace_survey_4_1365_patch_0.0_20.0_480

        1/2      3.67G      5.239      10.52      3.914     0.6953          9        640:  31%|███       | 73/238 [00:09<00:21,  7.83it/s]


batch ['/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_927_patch_760.0_0.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_515_patch_0.0_0.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_985_patch_0.0_380.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_861_patch_760.0_380.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_1072_patch_760.0_380.0_640_640.jpg', '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/images/global_birds_palmyra_CooperStrawn_53m_tile_clip_projected_739_patch_760.0_380.0_

KeyboardInterrupt: 

In [8]:
batch_idx = torch.Tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 3., 3., 4., 5., 6., 7.])
annotations_mask = [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False]

new_batch_idx = batch_idx[annotations_mask]
new_batch_idx


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.])

In [15]:
b = new_batch_idx.unique(return_inverse=True)
b[1]


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:

selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
selected_img.extend(random.choices(os.listdir(img_path + '/images/'), k=5))

# Predict results for randomly selected images
results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )



0: 640x640 2 birds, 1: 640x640 99 birds, 2: 640x640 6 birds, 3: 640x640 (no detections), 4: 640x640 1 bird, 121.8ms
0: 640x640 2 birds, 1: 640x640 99 birds, 2: 640x640 6 birds, 3: 640x640 (no detections), 4: 640x640 1 bird, 121.8ms
Speed: 1.3ms preprocess, 24.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.3ms preprocess, 24.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 23:
        print("block", i)
        print(a)

block 23
Conv_(
  (conv1): Conv2d(576, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(1, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
)


In [ ]:
model.subtask

'domainclassifier'

In [ ]:
print(B)

NameError: name 'B' is not defined

In [ ]:

module_path = os.path.abspath(os.path.join('..', '..'))
print(module_path)
module_path = module_path+'/runs'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

/vast/palmer/home.grace/eec42/BirdDetector
/vast/palmer/home.grace/eec42/BirdDetector/runs


In [ ]:
PRETRAINED_MODEL_PATH = module_path + '/detect/YOLO_pe_palmyra_10percentbkgd_test2/weights/best.pt'
model = YOLO('yolov8m.yaml', task='detect').load(PRETRAINED_MODEL_PATH)


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

In [ ]:
selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
for subdataset in ['global_birds_penguins', 'global_birds_palmyra']:
    selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=10))

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )


0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 (no detections), 19: 640x640 (no detections), 196.3ms
Speed: 1.5ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results[5].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (480, 480)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))

## Visualization

In [ ]:
TASK = 'deepcoral_detect' #detect
#model_name = 'deepcoral_test4'
#model_path = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'
#model = YOLO(model_path, TASK)
model_name = 'deepcoral_background_lscale16_epochs20_coralgain10'
PRETRAINED_MODEL_PATH = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'

model = YOLO(PRETRAINED_MODEL_PATH, TASK)

In [ ]:
datasets = ['source', 'target'] #['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins', 'global_birds_pfeifer']
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

In [ ]:
# FOR NORMAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'detect':
    selected_img = []
    for subdataset in datasets:
        selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=6))

    results = model.predict(
            #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
            source = [os.path.join(img_path + 'images/', img) for img in selected_img],
            conf = 0.2, 
            iou = 0.1,
            show=False,
            save=False
        )

In [ ]:
if TASK == 'detect':
    for img, result in zip(selected_img, results):

        detection_boxes = []
        save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
        for detect in range(len(result.boxes.cls)):
            det = {}
            det['conf'] = result.boxes.conf[detect].cpu()
            det['category'] = result.boxes.cls[detect].cpu()
            coords = result.boxes.xywhn[detect].cpu()
            det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
            detection_boxes.append(det)
            
        im_path = os.path.join(img_path + 'images/', img)
        visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                        confidence_threshold=0.0, detector_label_map=None,
                                        thickness=1,expansion=0, colormap=['Red'])

        selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
        if os.path.exists(selected_label):
            detection_boxes = []
            df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
            for irow, row in df.iterrows():  
                det = {}
                det['conf'] = None
                det['category'] = row[0]
                det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                detection_boxes.append(det)
        
            # Draw annotations
            save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
            visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['SpringGreen'])
                                            
            # Remove predictions-only images
            os.remove(save_path)


In [ ]:
# FOR DEEP CORAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'deepcoral_detect':

    for subdataset in datasets:
        selected_img = random.choices(os.listdir(img_path + subdataset + '/images/'), k=12)

        results = model.predict(
                #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
                source = [os.path.join(img_path, subdataset + '/images/', img) for img in selected_img],
                conf = 0.2, 
                iou = 0.1,
                show=False,
                save=False
            )
        
        for img, result in zip(selected_img, results):

            detection_boxes = []
            save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
            for detect in range(len(result.boxes.cls)):
                det = {}
                det['conf'] = result.boxes.conf[detect].cpu()
                det['category'] = result.boxes.cls[detect].cpu()
                coords = result.boxes.xywhn[detect].cpu()
                det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
                detection_boxes.append(det)
                
            im_path = os.path.join(img_path + subdataset + '/images/', img)
            visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['Red'])

            selected_label = img_path  + subdataset + '/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
            if os.path.exists(selected_label):
                detection_boxes = []
                df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
                for irow, row in df.iterrows():  
                    det = {}
                    det['conf'] = None
                    det['category'] = row[0]
                    det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                    detection_boxes.append(det)
        
                # Draw annotations
                save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
                visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                                confidence_threshold=0.0, detector_label_map=None,
                                                thickness=1,expansion=0, colormap=['SpringGreen'])
                
                # Remove predictions-only images
                os.remove(save_path)


In [ ]:
b

## YOLO Evaluation

In [ ]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.1, conf=0.2, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

In [ ]:
print(metrics.box.map)
print(metrics.box.map50)    # map50-95

## YOLO Evaluation per dataset

In [ ]:
datasets =  ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [ ]:
model_name = 'pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)
    print(metrics.box.map50)

# Change test folder to original name
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
data['test'] = 'test/images/'

## **TEST**

In [ ]:
model_name = 'pfeifer_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50_lr00.001_lrf0.0001'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

selected_img = (random.choices(os.listdir(img_path + '/images/'), k=1))
selected_img = [os.path.join(img_path + '/images/', x) for x in selected_img]
selected_img

In [ ]:
output = model(selected_img)

In [ ]:
output